# 模型参数的访问、初始化和共享

在[“线性回归的简洁实现”]一节中，我们通过`init`模块来初始化模型的全部参数。我们也介绍了访问模型参数的简单方法。本节将深入讲解如何访问和初始化模型参数，以及如何在多个层之间共享同一份模型参数。

我们先定义一个与上一节中相同的含单隐藏层的多层感知机。我们依然使用默认方式初始化它的参数，并做一次前向计算。

In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.1.0


In [2]:
net = tf.keras.models.Sequential()
net.add(tf.keras.layers.Flatten())
net.add(tf.keras.layers.Dense(256,activation=tf.nn.relu))
net.add(tf.keras.layers.Dense(10))

X = tf.random.uniform((2,20))
Y = net(X)
Y

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.17193761, -0.0922666 , -0.13595092, -0.11826225,  0.35631424,
         0.23411953,  0.18851405, -0.29137024, -0.07208098, -0.32998022],
       [ 0.04340871, -0.2755222 , -0.44214192, -0.02530663,  0.6582413 ,
         0.12312036,  0.24236704, -0.12245701, -0.3845298 , -0.5933588 ]],
      dtype=float32)>

## 4.2.1 access model parameters

对于使用`Sequential`类构造的神经网络，我们可以通过weights属性来访问网络任一层的权重。回忆一下上一节中提到的`Sequential`类与`tf.keras.Model`类的继承关系。对于`Sequential`实例中含模型参数的层，我们可以通过`tf.keras.Model`类的`weights`属性来访问该层包含的所有参数。下面，访问多层感知机`net`中隐藏层的所有参数。索引0表示隐藏层为`Sequential`实例最先添加的层。

In [3]:
net.weights[0], type(net.weights[0])

(<tf.Variable 'sequential/dense/kernel:0' shape=(20, 256) dtype=float32, numpy=
 array([[ 0.0481915 , -0.12631316,  0.01195522, ...,  0.02723822,
          0.02255017, -0.12760542],
        [ 0.0930911 , -0.01770672,  0.02268262, ...,  0.14521751,
          0.14541116, -0.06110913],
        [ 0.04290995, -0.11152898,  0.064651  , ..., -0.13156009,
         -0.09439682,  0.1101175 ],
        ...,
        [-0.09814546,  0.00927126, -0.11610402, ...,  0.04871327,
         -0.09315487, -0.05659153],
        [ 0.06200621, -0.09238889,  0.04176129, ..., -0.14487112,
          0.06835862, -0.06371358],
        [ 0.13075373, -0.04125882,  0.04096027, ...,  0.05943713,
         -0.11458447, -0.08245869]], dtype=float32)>,
 tensorflow.python.ops.resource_variable_ops.ResourceVariable)

## 4.2.2 initialize params

我们在[“数值稳定性和模型初始化”]一节中描述了模型的默认初始化方法：权重参数元素为[-0.07, 0.07]之间均匀分布的随机数，偏差参数则全为0。但我们经常需要使用其他方法来初始化权重。在下面的例子中，我们将权重参数初始化成均值为0、标准差为0.01的正态分布随机数，并依然将偏差参数清零。

In [4]:
class Linear(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.d1 = tf.keras.layers.Dense(
            units=10,
            activation=None,
            kernel_initializer=tf.random_normal_initializer(mean=0,stddev=0.01),
            bias_initializer=tf.zeros_initializer()
        )
        self.d2 = tf.keras.layers.Dense(
            units=1,
            activation=None,
            kernel_initializer=tf.ones_initializer(),
            bias_initializer=tf.ones_initializer()
        )

    def call(self, input):
        output = self.d1(input)
        output = self.d2(output)
        return output

net = Linear()
net(X)
net.get_weights()

[array([[ 4.51393833e-04,  1.57772638e-02, -1.86072791e-03,
          9.84685216e-03,  1.17519898e-02, -3.12424754e-03,
         -9.02889855e-03, -2.59149168e-02,  6.66950271e-03,
         -3.18642822e-03],
        [ 8.68504588e-03,  7.08601344e-03,  1.74540130e-03,
          1.08063575e-02, -1.17103141e-02, -3.71275260e-03,
         -5.68187982e-03,  1.98864192e-03, -5.00063365e-03,
         -1.03827775e-03],
        [-9.89197753e-03, -4.18905262e-03,  1.65334251e-02,
         -5.04891668e-03, -9.47635528e-03, -1.82326464e-03,
          1.91288700e-06, -1.32915629e-02,  1.11705707e-02,
         -1.58366859e-02],
        [ 1.27096614e-02,  1.15885818e-02, -2.00654278e-04,
         -8.13968387e-03,  3.31314979e-04,  1.26107298e-02,
         -6.44148933e-03, -7.69741414e-03, -3.03531438e-03,
         -1.46293696e-02],
        [ 2.77605117e-03, -6.10801065e-03, -2.19253893e-03,
         -3.63684585e-03,  1.28923450e-02, -2.79305899e-03,
         -1.15096420e-02,  7.75661319e-03,  5.740740

## 4.2.3 define initializer

可以使用`tf.keras.initializers`类中的方法实现自定义初始化。

In [5]:
def my_init():
    return tf.keras.initializers.Ones()

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, kernel_initializer=my_init()))

Y = model(X)
model.weights[0]

<tf.Variable 'sequential_1/dense_4/kernel:0' shape=(20, 64) dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>